<a href="https://colab.research.google.com/github/ValentinCord/HandsOnAI_2/blob/main/NLP_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [263]:
!/opt/bin/nvidia-smi
!rm -rf sample_data

Fri Dec  9 10:04:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    26W /  70W |   8380MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [264]:
!pip3 install transformers
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [265]:
# basics 
import os
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# transformers 
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from transformers import CamembertModel, CamembertTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# plot 
import matplotlib.pyplot as plt 
import seaborn as sns 

In [266]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [267]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [268]:
device

'cuda'

# Lecture et analyse pandas

In [269]:
train_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_train.csv'
test_path = '/content/drive/MyDrive/HandOnAI_2_NLP/fake_test.csv'

In [270]:
df = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [271]:
df = df.drop(['Unnamed: 0', 'target_name'], axis = 1)
df_test = df_test.drop(['Unnamed: 0', 'target_name'], axis = 1)

In [272]:
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('french'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    #text = REPLACE_BY_SPACE_RE.sub(' ', text)
    #text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [273]:
df['data'] = df['data'].apply(clean_text)
#df['data'] = df['data'].str.replace('\d+', '')

In [274]:
df.head()

,data,label
0,22e jour consécutif grève reconductible contre...,0
1,"depuis plusieurs mois, initiatives chercheurs ...",0
2,google vient d'introduire mise jour applicatio...,0
3,portrait. chacun s’empresse autour d’elle tand...,0
4,"« n’y a risque pénurie » carburant, a déclaré ...",0


In [275]:
def get_split(text1):
    l_total = []
    l_parcial = []
    if len(text1.split())//400 >0:
        n = len(text1.split())//400
    else: 
        n = 1
    for w in range(n):
        if w == 0:
            l_parcial = text1.split()[:450]
            l_total.append(" ".join(l_parcial))
        else:
            l_parcial = text1.split()[w*400:w*400 + 450]
            l_total.append(" ".join(l_parcial))
    return l_total

In [276]:
df['text_split'] = df['data'].apply(get_split)

In [277]:
for index, row in df.iterrows():
  if len(row['text_split']) > 1: 
    print(index)
    break

31


In [278]:
print(df['text_split'][31][0].split()[400:])
print(df['text_split'][31][1].split()[:100])

['océans', 'avant', 'après', 'crise', 'kt.', 'chercheur', 'northwestern', 'university', 'coquilles', 'fossilisée', 'collectée', 'lors', "d'un", 'voyage', 'recherche', 'terrain', "l'île", 'seymour,', 'antarctique.', '©', 'northwestern', 'university', 'effet', 'serre', 'acidification', 'océans', 'informations', 'fournies', 'analyses', 'conduites', 'méthode', 'spectrométrie', 'masse', ',', 'similaire', 'celles', 'utilisées', 'membres', 'lsce', 'france', 'locaux', 'ice', ',', 'permis', "d'établir", 'océans', 'subissaient', 'acidification', 'rapide', 'avant']
['océans', 'avant', 'après', 'crise', 'kt.', 'chercheur', 'northwestern', 'university', 'coquilles', 'fossilisée', 'collectée', 'lors', "d'un", 'voyage', 'recherche', 'terrain', "l'île", 'seymour,', 'antarctique.', '©', 'northwestern', 'university', 'effet', 'serre', 'acidification', 'océans', 'informations', 'fournies', 'analyses', 'conduites', 'méthode', 'spectrométrie', 'masse', ',', 'similaire', 'celles', 'utilisées', 'membres', 'l

In [279]:
train_l = []
label_l = []
index_l =[]
for idx,row in df.iterrows():
    for l in row['text_split']:
        train_l.append(l)
        label_l.append(row['label'])
        index_l.append(idx)
len(train_l), len(label_l), len(index_l)

(1568, 1568, 1568)

In [280]:
df = pd.DataFrame({'data':train_l, 'label':label_l})
df.head()

,data,label
0,22e jour consécutif grève reconductible contre...,0
1,"depuis plusieurs mois, initiatives chercheurs ...",0
2,google vient d'introduire mise jour applicatio...,0
3,portrait. chacun s’empresse autour d’elle tand...,0
4,"« n’y a risque pénurie » carburant, a déclaré ...",0


In [281]:
new_df=df

df_target=pd.DataFrame(new_df['label'])
df_target['temp']=0
for i in range(len(df_target)):
    if df_target['label'].loc[i]==0:
        df_target['temp'].loc[i]=1
new_df1=new_df.drop(['label'],axis=1)
new_df2=pd.DataFrame(df_target)

new_df3=pd.concat([new_df1,new_df2],axis = 1)
df=new_df3

df['label'] = df[df.columns[1:]].values.tolist()
new_df = df[['data', 'label']].copy()
df = new_df
df = df.rename(columns={'data': 'text'})
df.head()

,text,label
0,22e jour consécutif grève reconductible contre...,"[0, 1]"
1,"depuis plusieurs mois, initiatives chercheurs ...","[0, 1]"
2,google vient d'introduire mise jour applicatio...,"[0, 1]"
3,portrait. chacun s’empresse autour d’elle tand...,"[0, 1]"
4,"« n’y a risque pénurie » carburant, a déclaré ...","[0, 1]"


In [282]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len, is_target = True):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.max_len = max_len
        if is_target: 
          self.targets = self.data.label
        else: 
          self.targets = None

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        if self.targets is None: 
          return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(mask, dtype=torch.long),
              'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
          }
        else: 
          return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(mask, dtype=torch.long),
              'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
              'targets': torch.tensor(self.targets[index], dtype=torch.float)
          }

In [283]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 10
VALID_BATCH_SIZE = 10
EPOCHS = 10
LEARNING_RATE = 1e-05

In [284]:
!pip install sentencepiece

tranfo_name = "cmarkea/distilcamembert-base"
tokenizer = AutoTokenizer.from_pretrained(tranfo_name)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [285]:
df.head()

,text,label
0,22e jour consécutif grève reconductible contre...,"[0, 1]"
1,"depuis plusieurs mois, initiatives chercheurs ...","[0, 1]"
2,google vient d'introduire mise jour applicatio...,"[0, 1]"
3,portrait. chacun s’empresse autour d’elle tand...,"[0, 1]"
4,"« n’y a risque pénurie » carburant, a déclaré ...","[0, 1]"


In [286]:
df.tail()

,text,label
1563,« 15000 biographies consacrées président sorti...,"[1, 0]"
1564,mobilisation gilets jaunes continue soir. cert...,"[1, 0]"
1565,très célèbre très renommé stephen hawking a dé...,"[1, 0]"
1566,298 euros. c’est somme totale qu’aura dépensée...,"[1, 0]"
1567,ainsi françois pinault a annoncé premier temps...,"[1, 0]"


In [287]:
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state=200)
test_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (1568, 2)
TRAIN Dataset: (1254, 2)
TEST Dataset: (314, 2)


In [288]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [289]:
import torch

class BERTClass(torch.nn.Module):
    def __init__(self):
      super(BERTClass, self).__init__()
      self.l1 = CamembertModel.from_pretrained(tranfo_name)
      self.l3 = torch.nn.Linear(768, 2) #2 = binary classification
    
    def forward(self, ids, mask, token_type_ids):
      output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids)
      output = self.l3(output_1['pooler_output'])

      return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream

BERTClass(
  (l1): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(32005, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0): CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

In [290]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [291]:
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        token_type_ids = data['token_type_ids'].to(device)
        targets = data['targets'].to(device)
        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%10==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [292]:
for epoch in range(EPOCHS):
    train(epoch)

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  0.6794766783714294
Epoch: 0, Loss:  0.6667909622192383
Epoch: 0, Loss:  0.5596280097961426
Epoch: 0, Loss:  0.4625519812107086
Epoch: 0, Loss:  0.377626895904541
Epoch: 0, Loss:  0.17535008490085602
Epoch: 0, Loss:  0.23995564877986908
Epoch: 0, Loss:  0.5155369639396667
Epoch: 0, Loss:  0.3190128803253174
Epoch: 0, Loss:  0.08030527085065842
Epoch: 0, Loss:  0.09892450273036957
Epoch: 0, Loss:  0.22013838589191437
Epoch: 0, Loss:  0.09334536641836166
Epoch: 1, Loss:  0.09238988161087036
Epoch: 1, Loss:  0.047227513045072556
Epoch: 1, Loss:  0.08841341733932495
Epoch: 1, Loss:  0.19420547783374786
Epoch: 1, Loss:  0.056577395647764206
Epoch: 1, Loss:  0.03807767108082771
Epoch: 1, Loss:  0.3118326663970947
Epoch: 1, Loss:  0.04931376129388809
Epoch: 1, Loss:  0.3246093690395355
Epoch: 1, Loss:  0.043811406940221786
Epoch: 1, Loss:  0.32111474871635437
Epoch: 1, Loss:  0.7267404198646545
Epoch: 1, Loss:  0.04255778715014458
Epoch: 2, Loss:  0.040248412638902664
Epoch: 2

In [293]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
      for _, data in enumerate(testing_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)
        fin_targets.extend(targets.cpu().detach().numpy().tolist())
        fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [294]:
from sklearn import metrics
for epoch in range(EPOCHS):
  outputs, targets = validation(epoch)
  outputs = np.array(outputs) >= 0.5
  accuracy = metrics.accuracy_score(targets, outputs)
  f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
  f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
  print(f"Accuracy Score = {accuracy}")
  print(f"F1 Score (Micro) = {f1_score_micro}")
  print(f"F1 Score (Macro) = {f1_score_macro}")

tensor([[-6.1568,  6.2695],
        [ 6.1124, -6.1898],
        [ 5.9038, -6.0152],
        [ 5.5376, -5.6898],
        [-6.1640,  6.2537],
        [ 5.8591, -5.9936],
        [ 5.5990, -5.6349],
        [ 5.8801, -5.9730],
        [-6.0899,  6.2479],
        [ 5.8105, -5.9088]], device='cuda:0')
tensor([[ 5.7055, -5.8055],
        [-6.0901,  6.1977],
        [-6.1763,  6.2855],
        [ 6.1028, -6.1923],
        [ 5.8106, -5.8923],
        [ 6.1062, -6.2079],
        [-2.9801,  3.0152],
        [-6.2199,  6.3311],
        [ 6.0555, -6.1365],
        [-6.1968,  6.2745]], device='cuda:0')
tensor([[-6.1085,  6.1699],
        [-6.1266,  6.2792],
        [ 5.7119, -5.8503],
        [-6.2265,  6.3666],
        [-6.1850,  6.2840],
        [-5.5916,  5.7008],
        [ 4.4695, -4.5843],
        [-6.2507,  6.3880],
        [-6.1471,  6.2968],
        [-6.2469,  6.3595]], device='cuda:0')
tensor([[-6.1385,  6.2213],
        [-6.1150,  6.2067],
        [-6.0502,  6.2320],
        [-6.0966,  6.2

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tensor([[-6.1308,  6.2711],
        [-6.1797,  6.3098],
        [-6.1512,  6.2136],
        [ 5.6326, -5.7570],
        [ 3.4878, -3.6874],
        [-5.9310,  5.9462],
        [-2.7758,  2.6352],
        [ 5.9954, -6.1575],
        [ 5.8468, -5.8968],
        [ 5.7050, -5.6570]], device='cuda:0')
tensor([[-6.2507,  6.4135],
        [ 5.9318, -5.9732],
        [ 5.0110, -5.1105],
        [ 5.9967, -6.0744],
        [-5.9522,  6.0137],
        [-6.1150,  6.2067],
        [-1.8118,  1.7679],
        [-6.2146,  6.3206],
        [ 1.7028, -1.9577],
        [ 5.8105, -5.9088]], device='cuda:0')
tensor([[ 5.2662, -5.3748],
        [-5.9933,  6.0621],
        [ 6.1222, -6.2348],
        [-6.1828,  6.3119],
        [-6.2464,  6.3681],
        [-6.1665,  6.2695],
        [-6.1399,  6.2610],
        [-5.8892,  6.0010],
        [ 5.6577, -5.7884],
        [-5.8295,  5.9723]], device='cuda:0')
tensor([[-6.0668,  6.1482],
        [ 5.9776, -6.0469],
        [-6.2365,  6.3505],
        [-5.5261,  5.6

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tensor([[-6.2085,  6.3867],
        [-5.5821,  5.6602],
        [ 6.1534, -6.2588],
        [-6.1466,  6.2845],
        [-4.8498,  4.8851],
        [ 6.0580, -6.1454],
        [-6.0899,  6.2479],
        [-6.2441,  6.3640],
        [-6.0865,  6.1497],
        [-2.7758,  2.6352]], device='cuda:0')
tensor([[-6.2110,  6.3515],
        [-6.1595,  6.2370],
        [-6.1828,  6.3119],
        [-5.9054,  6.0547],
        [-6.1100,  6.2206],
        [ 5.3786, -5.4432],
        [ 5.9916, -6.0109],
        [-6.0283,  6.1229],
        [-6.2676,  6.3755],
        [-6.1642,  6.2148]], device='cuda:0')
tensor([[ 5.8820, -5.9295],
        [ 5.8707, -5.9293],
        [ 5.5794, -5.6049],
        [-6.1544,  6.2590],
        [-6.0975,  6.1940],
        [ 5.2662, -5.3748],
        [-6.2349,  6.2974],
        [ 6.0765, -6.1883],
        [-6.0471,  6.1945],
        [-6.1763,  6.2855]], device='cuda:0')
tensor([[ 5.5253, -5.7025],
        [-6.0653,  6.1625],
        [-5.9007,  5.9789],
        [ 6.1151, -6.1

...

tensor([[ 5.7443, -5.8133],
        [-5.1202,  5.1076],
        [-6.1448,  6.2493],
        [-6.2521,  6.3934],
        [ 5.9346, -6.0449],
        [ 5.5935, -5.6670],
        [ 2.3187, -2.5027],
        [-6.1385,  6.2213],
        [ 5.7119, -5.8503],
        [-5.9423,  6.0503]], device='cuda:0')
tensor([[ 6.0653, -6.1676],
        [ 4.4695, -4.5843],
        [-6.2154,  6.3270],
        [-5.8738,  5.9580],
        [ 5.9909, -6.0480],
        [ 5.9045, -5.9806],
        [-6.2365,  6.3505],
        [-6.0668,  6.1482],
        [-6.2202,  6.3937],
        [-6.1080,  6.1801]], device='cuda:0')
tensor([[-5.8640,  6.0175],
        [-6.2676,  6.3755],
        [ 6.0490, -6.1689],
        [-6.1921,  6.2811],
        [ 6.0753, -6.1517],
        [-5.8835,  6.0486],
        [ 2.5926, -2.7141],
        [ 3.5608, -3.6160],
        [ 5.8820, -5.9295],
        [-6.1907,  6.2921]], device='cuda:0')
tensor([[-6.2653,  6.3274],
        [-6.2336,  6.3308],
        [ 5.3786, -5.4432],
        [-0.1407,  0.3

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tensor([[ 5.9318, -5.9732],
        [-6.2455,  6.3808],
        [-6.1399,  6.2066],
        [-6.1653,  6.2800],
        [ 6.0555, -6.1365],
        [-6.1080,  6.1801],
        [ 5.8163, -5.9759],
        [ 5.8591, -5.9936],
        [ 3.4878, -3.6874],
        [-6.2507,  6.3880]], device='cuda:0')
tensor([[ 2.5926, -2.7141],
        [ 5.2662, -5.3748],
        [-6.1487,  6.3013],
        [-5.9310,  5.9462],
        [-6.0780,  6.2220],
        [-6.1747,  6.3023],
        [-6.1642,  6.2148],
        [-6.2199,  6.3311],
        [-6.1595,  6.2370],
        [-6.1980,  6.3101]], device='cuda:0')
tensor([[-6.0960,  6.1568],
        [ 5.8683, -5.9792],
        [ 4.9527, -5.0346],
        [-6.1628,  6.2546],
        [ 5.4435, -5.6201],
        [-5.1202,  5.1076],
        [ 5.9712, -6.0325],
        [-6.0152,  6.1755],
        [-5.9054,  6.0547],
        [-6.0711,  6.1839]], device='cuda:0')
tensor([[ 5.9909, -6.0480],
        [-5.8986,  6.0271],
        [ 4.8482, -4.9050],
        [ 5.8105, -5.9

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tensor([[-6.0152,  6.1755],
        [-6.1423,  6.2536],
        [-6.2126,  6.3992],
        [-6.0614,  6.1865],
        [ 6.1124, -6.1898],
        [-6.2412,  6.3673],
        [-6.1850,  6.2840],
        [ 0.2086, -0.3533],
        [ 6.1879, -6.2859],
        [ 5.5913, -5.6862]], device='cuda:0')
tensor([[-6.1412,  6.3026],
        [-6.0267,  6.1667],
        [-6.2436,  6.3980],
        [-6.1595,  6.2370],
        [-5.8485,  6.0427],
        [ 5.9858, -6.0753],
        [ 5.1660, -5.1469],
        [ 6.1028, -6.1923],
        [-6.2256,  6.3634],
        [ 5.9866, -6.0831]], device='cuda:0')
tensor([[-6.2469,  6.3595],
        [ 5.9526, -6.1210],
        [-6.1137,  6.2444],
        [ 5.9010, -6.0075],
        [-6.1100,  6.2206],
        [-6.1448,  6.2493],
        [-6.0960,  6.1568],
        [-6.2146,  6.3206],
        [ 5.9909, -6.0480],
        [-6.2507,  6.4135]], device='cuda:0')
tensor([[-6.2290,  6.3610],
        [-6.1568,  6.2695],
        [-6.1384,  6.2960],
        [-6.2521,  6.3

In [295]:
checkpoint = {'model': BERTClass(),
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

Some weights of the model checkpoint at cmarkea/distilcamembert-base were not used when initializing CamembertModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing CamembertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertModel were not initialized from the model checkpoint at cmarkea/distilcamembert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream

In [296]:
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    model = checkpoint['model']
    model.load_state_dict(checkpoint['state_dict'])
    for parameter in model.parameters():
        parameter.requires_grad = False
    
    model.eval()
    return model

In [297]:
the_model = load_checkpoint('checkpoint.pth')

In [298]:
news = "hello there"

d = {'text': [news]}
df = pd.DataFrame(data=d)

test_set = CustomDataset(df, tokenizer, MAX_LEN, is_target = False)

In [299]:
_params = {'batch_size': 1,
                'shuffle': True,
                'num_workers': 0
                }

_loader = DataLoader(test_set, **_params)

In [303]:
the_model.eval()
the_model.to(device)
for _,data in enumerate(_loader, 0):
  ids = data['ids'].to(device)
  mask = data['mask'].to(device)
  token_type_ids = data['token_type_ids'].to(device)
  
  outputs = the_model(ids, mask, token_type_ids)
  _, predicted = torch.max(outputs.data, 1)

  print(predicted)

tensor([[ 4.0219, -4.0397]], device='cuda:0')
tensor([0], device='cuda:0')


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
